In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
train_file = "./chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

In [6]:
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

In [7]:
MAX_SEQ_LEN = 15
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [8]:
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

In [9]:
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

In [10]:
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [11]:
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1

In [12]:
input_layer = Input(shape=(MAX_SEQ_LEN, ))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

In [15]:
conv1 = Conv1D(
    filters=128,
    kernel_size=3,
    padding='valid',
    activation=tf.nn.relu
)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters=128,
    kernel_size=4,
    padding='valid',
    activation=tf.nn.relu
)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters=128,
    kernel_size=5,
    padding='valid',
    activation=tf.nn.relu
)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)


In [16]:
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

In [17]:
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Train for 414 steps, validate for 119 steps
Epoch 1/5
414/414 [==============================] - 5s 13ms/step - loss: 0.8828 - accuracy: 0.5835 - val_loss: 0.5246 - val_accuracy: 0.8071
Epoch 2/5
414/414 [==============================] - 5s 12ms/step - loss: 0.5021 - accuracy: 0.8105 - val_loss: 0.2712 - val_accuracy: 0.9082
Epoch 3/5
414/414 [==============================] - 5s 12ms/step - loss: 0.2914 - accuracy: 0.8992 - val_loss: 0.1526 - val_accuracy: 0.9628
Epoch 4/5
414/414 [==============================] - 5s 12ms/step - loss: 0.1869 - accuracy: 0.9425 - val_loss: 0.1021 - val_accuracy: 0.9738
Epoch 5/5
414/414 [==============================] - 5s 12ms/step - loss: 0.1333 - accuracy: 0.9593 - val_loss: 0.0673 - val_accuracy: 0.9801


In [19]:
loss, accuracy = model.evaluate(test_ds, verbose=1)

print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

60/60 [==============================] - 0s 2ms/step - loss: 0.0479 - accuracy: 0.9890
Accuracy: 98.900169
loss: 0.047883


In [20]:
model.save('cnn_model.h5')